### Convert big tif files to smaller png files

In [19]:
import matplotlib.pyplot as plt
import os
import numpy as np
import skimage.external.tifffile as tiff
import cv2
import shutil

In [2]:
# path_to_masks = './stratford/masks/'                    # assuming big tif images are in masks folder
# path_to_save_converted_masks = './stratford/masks_png/'    # assuming png to be saved in layers folder

# images = os.listdir(path_to_masks)

# RESIZE_TO = (5000, 5000)

In [3]:
# for image in images:
#     img_ = tiff.imread(os.path.join(path_to_masks, image))
    
#     mask_resize_ = cv2.resize(img_.astype(np.uint8), (5000,5000))
    
#     file_name = image.split('mask_')[-1].replace('tif', 'png')
    
#     cv2.imwrite(os.path.join(path_to_save_converted_masks, file_name), mask_resize_)

### Make train_crops.txt and val_crops.txt


In [36]:
import mmap
from skimage import io
from matplotlib import image
import random
import pandas as pd

In [5]:
size = 1700
stride = 1500

VAL_SPLIT = 20
TEST_SPLIT = 10
random.seed(7)

image_extension = '.png'
mask_extension =  '.png'

In [6]:
def is_pad_needed(H, W, C, size, stride, img, image_name):
    if (H%stride) == 0 and (W%stride) == 0:
        print('No padding required for image {}'.format(image_name))
    
    else:
        print('Image -> {}:'.format(image_name))
        if (H%stride) != 0:
            print('\t Padded horizontally, {} pixels'.format(size - H%stride))
            h_pad = W + size - H%stride
        
        if (W%stride) != 0:
            print('\t Padded vertically  , {} pixels'.format(size - W%stride))
            v_pad = H + size - W%stride
        
        temp = np.zeros((v_pad, h_pad, C), dtype=np.uint8)
        temp[:H, :W, :] = img
        return temp

In [7]:
def get_valid_image_file(image_file):
    try:
        img = io.imread(image_file)
    except:
        return False
    else:
        return True

In [8]:
files = []
# failed_files = []

base_dir = './data/stratford/rgb_images/'
layers_dir = './data/stratford/masks_png/'
save_dir = './data/stratford/crops' + '_' + str(size) + '_' + str(stride)

for file in os.listdir(base_dir):
    if file.endswith(image_extension):                                             # <--------- Mention the big image extension here. 
        files.append(file)
        
random.shuffle(files)

nos_val = int(len(files)*VAL_SPLIT/100)
nos_test = int(len(files)*TEST_SPLIT/100)

train_images = files[nos_test + nos_val :]
val_images = files[:nos_val]
test_images = files[nos_val : nos_test + nos_val]

In [10]:
if not os.path.isdir(save_dir):
    os.makedirs(save_dir+'/train_crops'+'/gt')
    os.makedirs(save_dir+'/train_crops'+'/images')
    os.makedirs(save_dir+'/val_crops'+'/gt')
    os.makedirs(save_dir+'/val_crops'+'/images')
    os.makedirs(save_dir+'/test_crops'+'/gt')
    os.makedirs(save_dir+'/test_crops'+'/images')

    os.makedirs(save_dir+'/images')
    os.makedirs(save_dir+'/gt')

In [ ]:
png_1km17501048040SWOOP2015_0_0.png


'./data/stratford/crops_1700_1500/images/png_1km17501048040SWOOP2015_0_0.png'

In [22]:
os.path.join(save_dir,'train_crops', 'gt')

'./data/stratford/crops_1700_1500/train_crops/gt'

In [26]:
crop_file = open(os.path.join(save_dir,('train_crops' + '.txt')), 'w')
for file in train_images:
    path_to_image = os.path.join(base_dir,file)
    path_to_layer = os.path.join(layers_dir, file[4:])
    
    imgBGR = cv2.imread(path_to_image)
    mask = cv2.imread(path_to_layer)
    
    H,W,C = imgBGR.shape
    
    imgBGR = is_pad_needed(H, W, C, size, stride, imgBGR, file)
    mask = is_pad_needed(H, W, C, size, stride, mask, file)
    
    maxx = int(np.ceil((H-size)/stride))
    maxy = int(np.ceil((W-size)/stride))
    
    for x in range(maxx+1):
        for y in range(maxy+1):
            im_ = imgBGR[x*stride:x*stride + size,y*stride:y*stride + size,:]
            mask_ = mask[x*stride:x*stride + size,y*stride:y*stride + size,:] * 255
            
            cv2.imwrite(save_dir+'/train_crops/images/{}_{}_{}.png'.format(file[:-4],x,y),  im_)
            cv2.imwrite(save_dir+'/train_crops/gt/{}_{}_{}.png'.format(file[:-4],x,y),  mask_)
            
            crop_file.write('{}_{}_{}\n'.format(file[:-4],x,y))
        
crop_file.close()

Image -> png_1km17498047990SWOOP2015.png:
	 Padded horizontally, 1200 pixels
	 Padded vertically  , 1200 pixels
Image -> png_1km17498047990SWOOP2015.png:
	 Padded horizontally, 1200 pixels
	 Padded vertically  , 1200 pixels
Image -> png_1km17499048000SWOOP2015.png:
	 Padded horizontally, 1200 pixels
	 Padded vertically  , 1200 pixels
Image -> png_1km17499048000SWOOP2015.png:
	 Padded horizontally, 1200 pixels
	 Padded vertically  , 1200 pixels
Image -> png_1km17503047990SWOOP2015.png:
	 Padded horizontally, 1200 pixels
	 Padded vertically  , 1200 pixels
Image -> png_1km17503047990SWOOP2015.png:
	 Padded horizontally, 1200 pixels
	 Padded vertically  , 1200 pixels
Image -> png_1km17503048020SWOOP2015.png:
	 Padded horizontally, 1200 pixels
	 Padded vertically  , 1200 pixels
Image -> png_1km17503048020SWOOP2015.png:
	 Padded horizontally, 1200 pixels
	 Padded vertically  , 1200 pixels
Image -> png_1km17502048040SWOOP2015.png:
	 Padded horizontally, 1200 pixels
	 Padded vertically  , 1200

In [27]:
crop_file = open(os.path.join(save_dir,('val_crops' + '.txt')), 'w')
for file in val_images:
    path_to_image = os.path.join(base_dir,file)
    path_to_layer = os.path.join(layers_dir, file[4:])
    
    imgBGR = cv2.imread(path_to_image)
    mask = cv2.imread(path_to_layer)
    
    H,W,C = imgBGR.shape
    
    imgBGR = is_pad_needed(H, W, C, size, stride, imgBGR, file)
    mask = is_pad_needed(H, W, C, size, stride, mask, file)
    
    maxx = int(np.ceil((H-size)/stride))
    maxy = int(np.ceil((W-size)/stride))
    
    for x in range(maxx+1):
        for y in range(maxy+1):
            im_ = imgBGR[x*stride:x*stride + size,y*stride:y*stride + size,:]
            mask_ = mask[x*stride:x*stride + size,y*stride:y*stride + size,:] * 255
            
            cv2.imwrite(save_dir+'/val_crops/images/{}_{}_{}.png'.format(file[:-4],x,y),  im_)
            cv2.imwrite(save_dir+'/val_crops/gt/{}_{}_{}.png'.format(file[:-4],x,y),  mask_)
            
            crop_file.write('{}_{}_{}\n'.format(file[:-4],x,y))
crop_file.close()

Image -> png_1km17501048000SWOOP2015.png:
	 Padded horizontally, 1200 pixels
	 Padded vertically  , 1200 pixels
Image -> png_1km17501048000SWOOP2015.png:
	 Padded horizontally, 1200 pixels
	 Padded vertically  , 1200 pixels
Image -> png_1km17501048020SWOOP2015.png:
	 Padded horizontally, 1200 pixels
	 Padded vertically  , 1200 pixels
Image -> png_1km17501048020SWOOP2015.png:
	 Padded horizontally, 1200 pixels
	 Padded vertically  , 1200 pixels
Image -> png_1km17501048030SWOOP2015.png:
	 Padded horizontally, 1200 pixels
	 Padded vertically  , 1200 pixels
Image -> png_1km17501048030SWOOP2015.png:
	 Padded horizontally, 1200 pixels
	 Padded vertically  , 1200 pixels
Image -> png_1km17498048020SWOOP2015.png:
	 Padded horizontally, 1200 pixels
	 Padded vertically  , 1200 pixels
Image -> png_1km17498048020SWOOP2015.png:
	 Padded horizontally, 1200 pixels
	 Padded vertically  , 1200 pixels
Image -> png_1km17500048000SWOOP2015.png:
	 Padded horizontally, 1200 pixels
	 Padded vertically  , 1200

In [28]:
crop_file = open(os.path.join(save_dir,('test_crops' + '.txt')), 'w')
for file in test_images:
    path_to_image = os.path.join(base_dir,file)
    path_to_layer = os.path.join(layers_dir, file[4:])
    
    imgBGR = cv2.imread(path_to_image)
    mask = cv2.imread(path_to_layer)
    
    H,W,C = imgBGR.shape
    
    imgBGR = is_pad_needed(H, W, C, size, stride, imgBGR, file)
    mask = is_pad_needed(H, W, C, size, stride, mask, file)
    
    maxx = int(np.ceil((H-size)/stride))
    maxy = int(np.ceil((W-size)/stride))
    
    for x in range(maxx+1):
        for y in range(maxy+1):
            im_ = imgBGR[x*stride:x*stride + size,y*stride:y*stride + size,:]
            mask_ = mask[x*stride:x*stride + size,y*stride:y*stride + size,:]
            
            cv2.imwrite(save_dir+'/test_crops/images/{}_{}_{}.png'.format(file[:-4],x,y),  im_)
            cv2.imwrite(save_dir+'/test_crops/gt/{}_{}_{}.png'.format(file[:-4],x,y),  mask_)
            
            crop_file.write('{}_{}_{}\n'.format(file[:-4],x,y))
crop_file.close()

Image -> png_1km17501048040SWOOP2015.png:
	 Padded horizontally, 1200 pixels
	 Padded vertically  , 1200 pixels
Image -> png_1km17501048040SWOOP2015.png:
	 Padded horizontally, 1200 pixels
	 Padded vertically  , 1200 pixels
Image -> png_1km17502048020SWOOP2015.png:
	 Padded horizontally, 1200 pixels
	 Padded vertically  , 1200 pixels
Image -> png_1km17502048020SWOOP2015.png:
	 Padded horizontally, 1200 pixels
	 Padded vertically  , 1200 pixels
Image -> png_1km17503048000SWOOP2015.png:
	 Padded horizontally, 1200 pixels
	 Padded vertically  , 1200 pixels
Image -> png_1km17503048000SWOOP2015.png:
	 Padded horizontally, 1200 pixels
	 Padded vertically  , 1200 pixels


### Moving files to make stratford dataset for 

In [103]:
all_images_file = open(os.path.join(save_dir, ('all_crops_') + str(size) + '_' + str(stride) + '.txt'), mode = 'w')

In [104]:
def make_testing_dir(mode):
    names = pd.read_csv(os.path.join(save_dir,(mode + '.txt')), header = None)
    
    path_images = os.path.join(save_dir, mode, 'images')
    path_gt = os.path.join(save_dir, mode, 'gt')
    
    for i in range(len(names)):
        img_name = os.path.join(path_images, names.iloc[i,0]) + image_extension
        gt_name = os.path.join(path_gt, names.iloc[i,0]) + mask_extension
            
        shutil.copy2(src =  img_name, 
                     dst = os.path.join(save_dir, 'images'))   # moving image
        shutil.copy2(src =  gt_name, 
                     dst = os.path.join(save_dir, 'gt'))       # moving mask
    
        all_images_file.write(names.iloc[i,0] + image_extension + '\n')   # writing name to file

In [105]:
make_testing_dir('train_crops')
make_testing_dir('val_crops')
make_testing_dir('test_crops')

In [106]:
all_images_file.close()